## **** Viterbi ****

Submit:
*   Ilay Anais
*   Hadar Pur


# **** Imports ****

In [ ]:
import numpy as np
import itertools
import math
import enum

**Section D:**

Implement a program for the Viterbi algorithm on this HMM.

• Your program should receive a sequence over the alphabet A,C,G,T as input and produce the sequence of most likely annotations (hidden states).

• The transition and emission probabilities may be hard coded into the program, and you may also hard code the input DNA sequence, but make sure it is easy enough to change the input sequence when needed.

• The program should output the most probable sequence of states given the input sequence and the log-probability of the hidden state jointly with the data (log(P(X,S|HMM))).

• Make sure that your program performs calculations using log-probabilities (use natural log (ln()) as shown in Lecture #6).

• You may assume that the first base in the sequence is intergenic.

• Submit your documented code in an appendix file.

## **** Global vars ****


In [ ]:
S = 'CCATCGCACTCCGATGTGGCCGGTGCTCACGTTGCCT'

#Bases
Bases = {"A": 0, "C": 1, "G": 2, "T": 3}

#States
States = {"S1":0, "S2": 1, "S3": 2, "S4": 3, "S5": 4, "S6": 5}

#Emmissions matrix
model_emissions = np.zeros((len(States),len(Bases)))

#Transitions matrix
model_transitions = np.zeros((len(States),len(States)))

#Emmissions matrix
new_model_emissions = np.zeros((len(States),len(Bases)))

## **** HMM Model ****


In [ ]:
#Emmissions matrix setup
def set_emission_table():
  eps = 2.220446049250313e-16

  model_emissions[States['S1']][Bases['A']] = np.log(0.3)
  model_emissions[States['S1']][Bases['C']] = np.log(0.2)
  model_emissions[States['S1']][Bases['G']] = np.log(0.2)
  model_emissions[States['S1']][Bases['T']] = np.log(0.3)

  model_emissions[States['S2']][Bases['A']] = np.log(1)
  model_emissions[States['S2']][Bases['C']] = np.log(eps)
  model_emissions[States['S2']][Bases['G']] = np.log(eps)
  model_emissions[States['S2']][Bases['T']] = np.log(eps)

  model_emissions[States['S3']][Bases['A']] = np.log(eps)
  model_emissions[States['S3']][Bases['C']] = np.log(0.4)
  model_emissions[States['S3']][Bases['G']] = np.log(0.4)
  model_emissions[States['S3']][Bases['T']] = np.log(0.2)

  model_emissions[States['S4']][Bases['A']] = np.log(eps)
  model_emissions[States['S4']][Bases['C']] = np.log(0.4)
  model_emissions[States['S4']][Bases['G']] = np.log(0.4)
  model_emissions[States['S4']][Bases['T']] = np.log(0.2)

  model_emissions[States['S5']][Bases['A']] = np.log(eps)
  model_emissions[States['S5']][Bases['C']] = np.log(0.4)
  model_emissions[States['S5']][Bases['G']] = np.log(0.4)
  model_emissions[States['S5']][Bases['T']] = np.log(0.2)

  model_emissions[States['S6']][Bases['A']] = np.log(eps)
  model_emissions[States['S6']][Bases['C']] = np.log(eps)
  model_emissions[States['S6']][Bases['G']] = np.log(eps)
  model_emissions[States['S6']][Bases['T']] = np.log(1)

#Transitions matrix setup
def set_transition_table():
  eps = 2.220446049250313e-16
  
  model_transitions[States['S1']][States['S1']] = np.log(0.95)
  model_transitions[States['S1']][States['S2']] = np.log(0.05)
  model_transitions[States['S1']][States['S3']] = np.log(eps)
  model_transitions[States['S1']][States['S4']] = np.log(eps)
  model_transitions[States['S1']][States['S5']] = np.log(eps)
  model_transitions[States['S1']][States['S6']] = np.log(eps)

  model_transitions[States['S2']][States['S1']] = np.log(eps)
  model_transitions[States['S2']][States['S2']] = np.log(eps)
  model_transitions[States['S2']][States['S3']] = np.log(1)
  model_transitions[States['S2']][States['S4']] = np.log(eps)
  model_transitions[States['S2']][States['S5']] = np.log(eps)
  model_transitions[States['S2']][States['S6']] = np.log(eps)

  model_transitions[States['S3']][States['S1']] = np.log(eps)
  model_transitions[States['S3']][States['S2']] = np.log(eps)
  model_transitions[States['S3']][States['S3']] = np.log(eps)
  model_transitions[States['S3']][States['S4']] = np.log(1)
  model_transitions[States['S3']][States['S5']] = np.log(eps)
  model_transitions[States['S3']][States['S6']] = np.log(eps)

  model_transitions[States['S4']][States['S1']] = np.log(eps)
  model_transitions[States['S4']][States['S2']] = np.log(eps)
  model_transitions[States['S4']][States['S3']] = np.log(eps)
  model_transitions[States['S4']][States['S4']] = np.log(eps)
  model_transitions[States['S4']][States['S5']] = np.log(1)
  model_transitions[States['S4']][States['S6']] = np.log(eps)

  model_transitions[States['S5']][States['S1']] = np.log(eps)
  model_transitions[States['S5']][States['S2']] = np.log(eps)
  model_transitions[States['S5']][States['S3']] = np.log(0.8)
  model_transitions[States['S5']][States['S4']] = np.log(eps)
  model_transitions[States['S5']][States['S5']] = np.log(eps)
  model_transitions[States['S5']][States['S6']] = np.log(0.2)

  model_transitions[States['S6']][States['S1']] = np.log(0.95)
  model_transitions[States['S6']][States['S2']] = np.log(0.05)
  model_transitions[States['S6']][States['S3']] = np.log(eps)
  model_transitions[States['S6']][States['S4']] = np.log(eps)
  model_transitions[States['S6']][States['S5']] = np.log(eps)
  model_transitions[States['S6']][States['S6']] = np.log(eps)

print("Bases: ", Bases)

print("\nStates: ", States)

set_emission_table()
set_transition_table()

print("\nEmissions table: ")
print(model_emissions)

print("\nTransitions table: ")
print(model_transitions)

Bases:  {'A': 0, 'C': 1, 'G': 2, 'T': 3}

States:  {'S1': 0, 'S2': 1, 'S3': 2, 'S4': 3, 'S5': 4, 'S6': 5}

Emissions table: 
[[ -1.2039728   -1.60943791  -1.60943791  -1.2039728 ]
 [  0.         -36.04365339 -36.04365339 -36.04365339]
 [-36.04365339  -0.91629073  -0.91629073  -1.60943791]
 [-36.04365339  -0.91629073  -0.91629073  -1.60943791]
 [-36.04365339  -0.91629073  -0.91629073  -1.60943791]
 [-36.04365339 -36.04365339 -36.04365339   0.        ]]

Transitions table: 
[[ -0.05129329  -2.99573227 -36.04365339 -36.04365339 -36.04365339
  -36.04365339]
 [-36.04365339 -36.04365339   0.         -36.04365339 -36.04365339
  -36.04365339]
 [-36.04365339 -36.04365339 -36.04365339   0.         -36.04365339
  -36.04365339]
 [-36.04365339 -36.04365339 -36.04365339 -36.04365339   0.
  -36.04365339]
 [-36.04365339 -36.04365339  -0.22314355 -36.04365339 -36.04365339
   -1.60943791]
 [ -0.05129329  -2.99573227 -36.04365339 -36.04365339 -36.04365339
  -36.04365339]]


## **** Viterbi Algorithm ****


In [ ]:
def reconstruct_max_prob_annotation(seq, viterbi_matrix, viterbi_matrix_pointers):
  # Finding cell V[n,j] with highest value
  current_state = "S1"
  max_prob_annotation = -np.inf

  for state in States:
    prob = viterbi_matrix[States[state], len(seq)-1]
    if prob > max_prob_annotation:
      max_prob_annotation = prob
      current_state = state
  
  #Traceback from max_prob_annotation
  annotation = "" + current_state
  emission = "" + seq[-1]
  for i in reversed(range(1, len(seq))):
    current_state = viterbi_matrix_pointers[States[current_state], i]
    annotation = current_state + " " + annotation
    emission = seq[i-1] + "  " +  emission

  print("\nViterbi's max-prob annotation")
  print(annotation)
  print(emission)
  print("max log(P(S|X,HMM)) = ", max_prob_annotation)
  print("max P(S|X,HMM) = ", math.exp(max_prob_annotation))


def viterbiAlg(seq, emissions_matrix, transitions_matrix):
  # initialize viterbi matrix
  viterbi_matrix = np.zeros((len(States),len(seq)))
  viterbi_matrix_pointers = np.ndarray((len(States), len(seq)), dtype = object)

  for j in States:
    viterbi_matrix[States[j]][0] = -np.inf

  viterbi_matrix[States['S1']][0] = emissions_matrix[States['S1'], Bases[seq[0]]] 
  viterbi_matrix_pointers[States['S1']][0] = 'S1'

  for i in range(1, len(seq)):
    for j in States:
      viterbi_matrix[States[j], i] = max([viterbi_matrix[States[l], i-1] + transitions_matrix[States[l], States[j]] for l in States])
      viterbi_matrix[States[j], i] += emissions_matrix[States[j], Bases[seq[i]]]

      for k in States:
        viterb = viterbi_matrix[States[k], i-1]
        transition = transitions_matrix[States[k], States[j]]
        emmision = emissions_matrix[States[j], Bases[seq[i]]]
        
        if viterbi_matrix[States[j], i] == viterb + transition + emmision:
          viterbi_matrix_pointers[States[j]][i] = k

  return reconstruct_max_prob_annotation(seq, viterbi_matrix, viterbi_matrix_pointers)

**Section E:**

Run your program on the sequence specified in (a) above. Which of the possible paths is the most likely, and what is its log-probability?

In [ ]:
viterbiAlg(S, model_emissions, model_transitions)


Viterbi's max-prob annotation
S1 S1 S1 S1 S1 S1 S1 S1 S1 S1 S1 S1 S1 S2 S3 S4 S5 S3 S4 S5 S3 S4 S5 S3 S4 S5 S6 S1 S2 S3 S4 S5 S3 S4 S5 S3 S4
C  C  A  T  C  G  C  A  C  T  C  C  G  A  T  G  T  G  G  C  C  G  G  T  G  C  T  C  A  C  G  T  T  G  C  C  T
max log(P(S|X,HMM)) =  -52.77840110563882
max P(S|X,HMM) =  1.1984823322410419e-23


## **** Posterior-decoding Algorithm for HMM ****


**Section F:**

Implement a program that computes a posterior-decoding for this HMM.

• Your program should receive a sequence over the alphabet A,C,G,T as input and produce for each position i along the sequence the HMM state s with highest posterior probability for that position: P(Si=s|X,HMM).

• The program should print for each position the state and its posterior conditional probability (values should sum to 1 across states), and also print the log likelihood for the entire sequence (log(P(X|HMM))).

• Do this by implementing procedures for computing the forward and backward matrices, and follow the coding guidelines specified in (d) above. Namely, make sure that your program performs calculations using log- probabilities.

• Make use of the technique presented in Lecture #6 to compute log-of-sum- of-exponents. Submit your documented code in an appendix file.

Debugging tip:

• Start by implementing a version of the forward and backward algorithms that use standard probability calculations (without using the log-of-sum-of- exponents trick).

• Validate your calculations by making sure that the same value of the likelihood is retrieved by taking the inner product of every column of the Forward matrix with its counterpart in the Backward matrix.

• Apply the log-of-sum-of-exponents trick to each algorithm and make sure that the obtained values are logs of the original ones.

In [ ]:
def forwardAlg(seq, emissions_matrix, transitions_matrix):
  # forward matrix
  forward_matrix = np.zeros((len(States),len(seq)))
  
  # f[0,j] = -∞ for ,j>0
  for j in States:
    forward_matrix[States[j]][0] = -np.inf

  forward_matrix[States['S1']][0] = emissions_matrix[States['S1'], Bases[seq[0]]]

  for i in range(1, len(seq)):
    for j in States:
      a_max = max([forward_matrix[States[l], i-1] + transitions_matrix[States[l], States[j]] for l in States])

      for k in States:
        a_k = forward_matrix[States[k], i-1] + transitions_matrix[States[k], States[j]]
        b_k = a_k - a_max
        forward_matrix[States[j], i] += np.exp(b_k)
      
      #log(∑l exp(al)) = log(∑l exp(amax + bl )) = log(exp(amax)∑l exp(bl)) = amax + log(∑l exp(bl))
      forward_matrix[States[j], i] = np.log(forward_matrix[States[j], i]) + a_max + emissions_matrix[States[j], Bases[seq[i]]] 
  
  return forward_matrix

def backwardAlg(seq, emissions_matrix, transitions_matrix):
  # backward matrix
  backward_matrix = np.zeros((len(States),len(seq)))

  for i in range(len(seq) - 2, -1, -1):
    for j in States:
      a_max = max([backward_matrix[States[l], i+1] + transitions_matrix[States[j], States[l]] + emissions_matrix[States[l], Bases[seq[i+1]]]for l in States])

      for k in States:
        a_k = backward_matrix[States[k], i+1] + transitions_matrix[States[j], States[k]] + emissions_matrix[States[k], Bases[seq[i+1]]]
        b_k = a_k - a_max
        backward_matrix[States[j], i] += np.exp(b_k)

      #log(∑l exp(al)) = log(∑l exp(amax + bl )) = log(exp(amax)∑l exp(bl)) = amax + log(∑l exp(bl))
      backward_matrix[States[j], i] = np.log(backward_matrix[States[j], i]) + a_max
  
  return backward_matrix

def likelihoodAlgForward(seq, forward_matrix):
  print("\nlikelihoodAlgForward")
  max_likelihoond = 0
  a_max = max([forward_matrix[States[j], len(seq) - 1] for j in States])

  for k in States:
    a_k = forward_matrix[States[k], len(seq) - 1]
    b_k = a_k - a_max
    max_likelihoond += np.exp(b_k)

  max_likelihoond = np.log(max_likelihoond) + a_max
  print("likelihood log(P(X|HMM)) = ", max_likelihoond)
  print("likelihood P(X|HMM) = ", math.exp(max_likelihoond))

def likelihoodAlgBackward(seq, backward_matrix, transitions_matrix, emissions_matrix):
  print("\nlikelihoodAlgBackward")
  max_likelihoond = 0
    
  for j in range(1, len(States)):
    backward_matrix[j][0] = -np.inf

  a_max = max([backward_matrix[States[j], 0] + emissions_matrix[States[j], Bases[seq[0]]] for j in States])
  for k in States:
    a_k = backward_matrix[States[k], 0] + emissions_matrix[States[k], Bases[seq[0]]]
    b_k = a_k - a_max
    max_likelihoond += np.exp(b_k)

  max_likelihoond = np.log(max_likelihoond) + a_max
  print("likelihood log(P(X|HMM)) = ", max_likelihoond)
  print("likelihood P(X|HMM) = ", math.exp(max_likelihoond))

def computeMaximumAPosterioriProbability(seq, forward_matrix, backward_matrix):
  # Objective: for a given sequence of observed of symbols X = X1...Xn and index i=1..n, compute the probability P(X , Si=sj |HMM) = ∑S|Si=sj P(X,S|HMM) for every sj
  # We want to sum over the probabilities of all paths in the decoding matrix that pass through cell [i,j]
  print("\nMaximum A-Posteriori Probability:")

  annotation = ""
  emission = ""
  probabilities = ""

  for i in range(len(seq) - 1, -1, -1):
    max_state = 'S1'
    max_prob = -np.inf

    # find max
    for j in States:
      prob = forward_matrix[States[j]][i] + backward_matrix[States[j]][i]
      if prob > max_prob:
        max_prob = prob
        max_state = j

    annotation = max_state + " " + annotation
    emission = seq[i] + "  " + emission

    prob_score = np.exp(forward_matrix[States[max_state]][i]) * np.exp(backward_matrix[States[max_state]][i])
    total_prob_score = 0

    for k in States:
      total_prob_score += np.exp(forward_matrix[States[k]][i]) * np.exp(backward_matrix[States[k]][i])

    prob_score = prob_score / total_prob_score
    probabilities = f"P(S_{i+1} = {max_state}|X,HMM) = {prob_score}\n" + probabilities

  print(probabilities)
  print("\n")
  print(annotation)
  print(emission)


**Section G:**

Run your program from (f) on the sequences pecified in (a) above.

Whatisthe most likely state for each position in the sequence? Compare your results to the ones obtained in (e).

In [ ]:
forwardMat = forwardAlg(S, model_emissions, model_transitions) 
# print("forward matrix: ", forwardMat)
backwardMat = backwardAlg(S, model_emissions, model_transitions) 
# print("\nbackward matrix: ", backwardMat)
likelihoodAlgForward(S, forwardMat)
likelihoodAlgBackward(S, backwardMat, model_transitions, model_emissions)
computeMaximumAPosterioriProbability(S, forwardMat, backwardMat)



likelihoodAlgForward
likelihood log(P(X|HMM)) =  -51.77951793226804
likelihood P(X|HMM) =  3.2541763644144575e-23

likelihoodAlgBackward
likelihood log(P(X|HMM)) =  -51.77951793226803
likelihood P(X|HMM) =  3.254176364414504e-23

Maximum A-Posteriori Probability:
P(S_1 = S1|X,HMM) = 1.0
P(S_2 = S1|X,HMM) = 1.0
P(S_3 = S1|X,HMM) = 0.9999999999999982
P(S_4 = S1|X,HMM) = 0.9999999999999974
P(S_5 = S1|X,HMM) = 0.9999999999999983
P(S_6 = S1|X,HMM) = 0.9999999999999986
P(S_7 = S1|X,HMM) = 0.9999999999999986
P(S_8 = S1|X,HMM) = 0.999999999999983
P(S_9 = S1|X,HMM) = 0.9999999999999827
P(S_10 = S1|X,HMM) = 0.9999999999999819
P(S_11 = S1|X,HMM) = 0.9999999999999847
P(S_12 = S1|X,HMM) = 0.9999999999999847
P(S_13 = S1|X,HMM) = 0.999999999999985
P(S_14 = S2|X,HMM) = 0.9612537613310652
P(S_15 = S3|X,HMM) = 0.9612537613310734
P(S_16 = S4|X,HMM) = 0.9612537613310747
P(S_17 = S5|X,HMM) = 0.9612537613310755
P(S_18 = S3|X,HMM) = 0.9612537613310772
P(S_19 = S4|X,HMM) = 0.9612537613310779
P(S_20 = S5|X,HM

**Section H:**

After further study of this virus, the scientists discovered that in some occasions its protein coding genes do contain A’s. 

So, they changed the model to allow A’s in genes with frequency of 5%, and T’s with frequency 15%. Other aspects of the model remained unchanged. 

Change your programs from (d) and (f) to reflect this model change and re-run them on the sequence specified in (a). Report the results. 

Did anything change?


## **** New HMM Model ****


In [ ]:
#Emmissions matrix setup
def set_new_emission_table():
  eps = 2.220446049250313e-16

  new_model_emissions[States['S1']][Bases['A']] = np.log(0.3)
  new_model_emissions[States['S1']][Bases['C']] = np.log(0.2)
  new_model_emissions[States['S1']][Bases['G']] = np.log(0.2)
  new_model_emissions[States['S1']][Bases['T']] = np.log(0.3)

  new_model_emissions[States['S2']][Bases['A']] = np.log(1)
  new_model_emissions[States['S2']][Bases['C']] = np.log(eps)
  new_model_emissions[States['S2']][Bases['G']] = np.log(eps)
  new_model_emissions[States['S2']][Bases['T']] = np.log(eps)

  new_model_emissions[States['S3']][Bases['A']] = np.log(0.05)
  new_model_emissions[States['S3']][Bases['C']] = np.log(0.4)
  new_model_emissions[States['S3']][Bases['G']] = np.log(0.4)
  new_model_emissions[States['S3']][Bases['T']] = np.log(0.15)

  new_model_emissions[States['S4']][Bases['A']] = np.log(0.05)
  new_model_emissions[States['S4']][Bases['C']] = np.log(0.4)
  new_model_emissions[States['S4']][Bases['G']] = np.log(0.4)
  new_model_emissions[States['S4']][Bases['T']] = np.log(0.15)

  new_model_emissions[States['S5']][Bases['A']] = np.log(0.05)
  new_model_emissions[States['S5']][Bases['C']] = np.log(0.4)
  new_model_emissions[States['S5']][Bases['G']] = np.log(0.4)
  new_model_emissions[States['S5']][Bases['T']] = np.log(0.15)

  new_model_emissions[States['S6']][Bases['A']] = np.log(eps)
  new_model_emissions[States['S6']][Bases['C']] = np.log(eps)
  new_model_emissions[States['S6']][Bases['G']] = np.log(eps)
  new_model_emissions[States['S6']][Bases['T']] = np.log(1)

#Transitions matrix initialize again
model_transitions = np.zeros((len(States),len(States)))

print("Bases: ", Bases)

print("\nStates: ", States)

set_new_emission_table()
set_transition_table()

print("\nEmissions table: ")
print(new_model_emissions)

print("\nTransitions table: ")
print(model_transitions)


viterbiAlg(S, new_model_emissions, model_transitions)

forwardMat = forwardAlg(S, new_model_emissions, model_transitions) 
# print("forward matrix: ", forwardMat)
backwardMat = backwardAlg(S, new_model_emissions, model_transitions) 
# print("\nbackward matrix: ", backwardMat)
likelihoodAlgForward(S, forwardMat)
likelihoodAlgBackward(S, backwardMat, model_transitions, new_model_emissions)

computeMaximumAPosterioriProbability(S, forwardMat, backwardMat)



Bases:  {'A': 0, 'C': 1, 'G': 2, 'T': 3}

States:  {'S1': 0, 'S2': 1, 'S3': 2, 'S4': 3, 'S5': 4, 'S6': 5}

Emissions table: 
[[ -1.2039728   -1.60943791  -1.60943791  -1.2039728 ]
 [  0.         -36.04365339 -36.04365339 -36.04365339]
 [ -2.99573227  -0.91629073  -0.91629073  -1.89711998]
 [ -2.99573227  -0.91629073  -0.91629073  -1.89711998]
 [ -2.99573227  -0.91629073  -0.91629073  -1.89711998]
 [-36.04365339 -36.04365339 -36.04365339   0.        ]]

Transitions table: 
[[ -0.05129329  -2.99573227 -36.04365339 -36.04365339 -36.04365339
  -36.04365339]
 [-36.04365339 -36.04365339   0.         -36.04365339 -36.04365339
  -36.04365339]
 [-36.04365339 -36.04365339 -36.04365339   0.         -36.04365339
  -36.04365339]
 [-36.04365339 -36.04365339 -36.04365339 -36.04365339   0.
  -36.04365339]
 [-36.04365339 -36.04365339  -0.22314355 -36.04365339 -36.04365339
   -1.60943791]
 [ -0.05129329  -2.99573227 -36.04365339 -36.04365339 -36.04365339
  -36.04365339]]

Viterbi's max-prob annotation
S